# Examen del 2do Parcial
Rolando Jiménez Sánchez

1. Utilizando el dataset de _Coactivation_matrix.mat_:
- Calcule el coeficiente de mundo pequeño
- Calcule las comunidades del grafo
- Calcule los hub
- Calcule la eficiencia global
- Calcule la eficiencia local
- Determine el grado de cada nodo

2. Utilizando el dataset de _Coactivation_matrix.mat_:
- Generar el mapa de calor de cada matriz de conectividad
- Generar la distribución de datos de cada matriz de conectividad
- ¿Qué valor se encuentra en el percentil 0.25, 0.5 y 0.75 de la matriz de conectividad?
- Cree los 3 grafos 2D filtrando la matriz de conectividad con los valores dados por los percentiles del ejercicio anterior

3. Utilizando el dataset de _Coactivation_matrix.mat_:
- Genere el grafo 3D
- Haga que el tamaño de nodos sea proporcional a su grado; es decir, q entre mayor sea su grado, mayor sea el tamaño del nodo ploteado
- Haga que el color de las aristas este relacionado al valor de la matriz de conectividad. Utilizar el map color Hot

4. Utilizando el dataset de _chb01_01.edf_:
- Calcule el coeficiente de mundo pequeño
- Calcule las comunidades del grafo
- Calcule los hub
- Calcule la eficiencia global
- Calcule la eficiencia local
- Determine el grado de cada nodo

5. Utilizando el dataset de _chb01_01.edf_:
- Genere el grafo 3D
- Haga que el tamaño de nodos sea proporcional a su grado; es decir, q entre mayor sea su grado, mayor sea el tamaño del nodo ploteado
- Haga que el color de las aristas este relacionado al valor de la matriz de conectividad. Utilizar el map color Hot


In [1]:
import matplotlib.pyplot as plt
import scipy.io
import seaborn as sns
import numpy as np
import pandas as pd
import networkx as nx
import scipy.io as sio
import networkx as nx
import seaborn as sns
from scipy.signal import coherence, hilbert
import scipy.io

# EJERCICIO 1

In [ ]:

Mat_Coact = 'C:/Users/rayom/OneDrive/Documentos/GitHub/Neurociencias-2026-1/S03_datasets/BCT/Coactivation_matrix.mat'
datos = scipy.io.loadmat(Mat_Coact)
data = datos["Coactivation_matrix"]
G = nx.from_numpy_array(data)
C = nx.average_clustering(G)
L = nx.average_shortest_path_length(G)
C, L
print(C)
print(L)

#Comunidades
from networkx.algorithms import community

comunidades = list(community.greedy_modularity_communities(G))
print(f"Número de comunidades: {len(comunidades)}")

for i, c in enumerate(comunidades):
    print(f"Comunidad {i+1} ({len(c)} nodos): {sorted(list(c))[:10]}")

#Hubs
degrees = dict(G.degree())
avg_degree = np.mean(list(degrees.values()))
hubs = [node for node, degree in degrees.items() if degree > avg_degree]
print("Potential Hubs (degree > average):", hubs)

# Eficiencia Global
def global_efficiency(G):
    n = len(G)
    if n <= 1:
        return 0
    lengths = dict(nx.all_pairs_shortest_path_length(G))
    s = 0
    for u in lengths:
        for v, d in lengths[u].items():
            if u != v:
                s += 1 / d
    return s / (n * (n - 1))

#Eficencia Local
def local_efficiency(G):
    effs = []
    for v in G:
        vecinos = list(G.neighbors(v))
        if len(vecinos) >= 2:
            sub = G.subgraph(vecinos)
            effs.append(global_efficiency(sub))
    return np.mean(effs)

E_global = global_efficiency(G)
E_local = local_efficiency(G)

print(f"Eficiencia global: {E_global:.3f}")
print(f"Eficiencia local:  {E_local:.3f}")

# Grado de cada nodo

degree_dict = dict(G.degree())
degree_dict


0.3844533292242755
2.2148737961545844
Número de comunidades: 3
Comunidad 1 (334 nodos): [0, 1, 2, 3, 4, 8, 9, 11, 13, 14]
Comunidad 2 (201 nodos): [5, 6, 7, 10, 12, 15, 16, 18, 20, 37]
Comunidad 3 (103 nodos): [26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
Potential Hubs (degree > average): [6, 7, 11, 16, 18, 19, 20, 22, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 50, 62, 63, 65, 68, 69, 70, 73, 76, 79, 80, 87, 93, 95, 97, 98, 100, 103, 104, 116, 118, 120, 121, 124, 125, 126, 128, 129, 130, 131, 135, 154, 158, 160, 167, 184, 186, 193, 194, 197, 202, 208, 211, 217, 219, 220, 223, 228, 230, 231, 232, 233, 235, 237, 238, 240, 244, 248, 252, 253, 259, 260, 261, 262, 263, 266, 267, 268, 269, 270, 271, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 288, 289, 291, 292, 296, 298, 303, 305, 308, 309, 327, 328, 329, 330, 331, 332, 333, 334, 335, 339, 344, 345, 346, 347, 348, 350, 351, 352, 353, 354, 356, 357, 358, 359, 360, 361, 362, 364, 365, 367, 368, 369, 370, 371, 373, 374, 375, 377, 383,

# EJERCICIO 2

In [ ]:
try:
    Mat_Coact = 'C:/Users/rayom/OneDrive/Documentos/GitHub/Neurociencias-2026-1/S03_datasets/BCT/Coactivation_matrix.mat'
    datos = scipy.io.loadmat(Mat_Coact)
    data = datos["Coactivation_matrix"]

    if data.shape[0] < data.shape[1]:
        data = data.T
        print(f"Datos cargados y transpuestos a formato (N_canales={data.shape[0]}, N_muestras={data.shape[1]})")

except FileNotFoundError:
    print("Error: Asegúrate de que 'Coactivation_matrix.mat' esté en el directorio correcto.")

# Parámetros de la señal
sfreq_target = 256
fmin, fmax = 8, 13
n_channels = data.shape[0]
n_samples = data.shape[1]

print(f"Analizando {n_channels} canales y {n_samples} muestras...")

#Matrices de conectividad

# Pearson
corr_matrix = np.corrcoef(data)
corr_df = pd.DataFrame(corr_matrix)

#Coherencia
coh_matrix = np.zeros((n_channels, n_channels))
for i in range(n_channels):
    for j in range(i, n_channels):
        with np.errstate(divide='ignore', invalid='ignore'):
            f, Cxy = coherence(data[i], data[j], fs=sfreq_target, nperseg=sfreq_target * 2)

        mask = (f >= fmin) & (f <= fmax)

        coh_val = np.mean(Cxy[mask])
        if np.isnan(coh_val):
            coh_val = 0.0

        coh_matrix[i, j] = coh_val
        coh_matrix[j, i] = coh_val

# Rellenar la diagonal con 1.0 (coherencia perfecta consigo mismo)
np.fill_diagonal(coh_matrix, 1.0)
coh_df = pd.DataFrame(coh_matrix)

# C. Phase Locking Value (PLV)
analytic_signal = hilbert(data)
phase_data = np.angle(analytic_signal)

plv_matrix = np.zeros((n_channels, n_channels))
for i in range(n_channels):
    for j in range(i, n_channels): # Optimización
        phase_diff = phase_data[i] - phase_data[j]
        # PLV es el valor absoluto de la fase media compleja
        plv_val = np.abs(np.sum(np.exp(1j * phase_diff)) / phase_diff.size)

        plv_matrix[i, j] = plv_val
        plv_matrix[j, i] = plv_val # Simetría

# Rellenar la diagonal con 1.0
np.fill_diagonal(plv_matrix, 1.0)
plv_df = pd.DataFrame(plv_matrix)

# Diccionario de las matrices para el bucle de análisis
matrices = {
    'Pearson_Correlation': corr_df,
    'Coherence': coh_df,
    'PLV': plv_df
}

Analizando 638 canales y 638 muestras...


In [ ]:
resultados_percentiles = {}
grafos_filtrados = {}

for name, df in matrices.items():
    print(f"\n{'='*50}")
    print(f"INICIANDO ANÁLISIS para: {name}")
    print(f"{'='*50}")


    plt.figure(figsize=(8, 8))
    sns.heatmap(df.values, annot=False, cmap='viridis' if name == 'PLV' else 'coolwarm',
                fmt=".2f", cbar_kws={'label': f'{name} Value'})
    plt.title(f'Mapa de Calor: {name} Matrix')
    plt.xlabel('Canal J')
    plt.ylabel('Canal I')
    plt.show()

    val_data = df.values[np.triu_indices_from(df, k=1)]

In [ ]:

    plt.figure(figsize=(8, 6))
    plt.hist(val_data, bins=50, color='skyblue', edgecolor='black')
    plt.title(f'Distribución de Datos: {name} Matrix')
    plt.xlabel(f'{name} Value')
    plt.ylabel('Frecuencia')
    plt.grid(axis='y', alpha=0.5)
    plt.show()


    p25 = np.percentile(val_data, 25)
    p50 = np.percentile(val_data, 50)
    p75 = np.percentile(val_data, 75)

    resultados_percentiles[name] = {'P25': p25, 'P50': p50, 'P75': p75}

    print(f"\nValores de Percentiles para {name}:")
    print(f"  Percentil 25 (Q1): {p25:.4f}")
    print(f"  Percentil 50 (Mediana): {p50:.4f}")
    print(f"  Percentil 75 (Q3): {p75:.4f}")


    percentiles = {'P25': p25, 'P50': p50, 'P75': p75}

    for p_name, threshold in percentiles.items():
        G_filt = nx.Graph()
        num_nodes = df.shape[0]
        G_filt.add_nodes_from(range(num_nodes))

        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                weight = df.iloc[i, j]
                if abs(weight) > threshold:
                    G_filt.add_edge(i, j, weight=weight)

        graph_key = f"{name}_{p_name}"
        grafos_filtrados[graph_key] = G_filt
        print(f"  Grafo '{p_name}': Creado con {G_filt.number_of_edges()} aristas.")


In [ ]:
#Grafos
print(f"\n{'='*50}")
print("GENERANDO GRÁFICOS 2D DE LOS 9 GRAFOS FILTRADOS")
print(f"{'='*50}")

for graph_key, G_to_draw in grafos_filtrados.items():
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G_to_draw, seed=42)

    edge_count = G_to_draw.number_of_edges()
    node_size = 20 if edge_count > 1000 else 50
    alpha_val = 0.3 if edge_count > 5000 else 0.6

    nx.draw(G_to_draw, pos, with_labels=False, node_size=node_size,
            edge_color='gray', alpha=alpha_val, width=0.5)
    plt.title(f'Grafo Filtrado ({G_to_draw.number_of_edges()} Aristas): {graph_key.replace("_", " ").title()}',
              fontsize=14)
    plt.show()

print("\nAnálisis completado.")

In [ ]:
 #Coherencia espectral
plt.figure(figsize=(8, 2))
coh = sns.heatmap(coh_df.values,
                 annot=False, cmap='GnBu', fmt=".2f")
plt.show()

plt.figure(figsize=(8, 6))
val_coh = corr_df.values[np.triu_indices_from(coh_df, k=1)]
plt.hist(val_coh, bins=45)

p25_c = np.percentile(val_coh, 25)
p50_c = np.percentile(val_coh, 50)
p75_c = np.percentile(val_coh, 75)

print(f"Percentil 25: {p25_c:.4f}")
print(f"Percentil 50: {p50_c:.4f}")
print(f"Percentil 75: {p75_c:.4f}")

# Ggráficas
percentiles_c = {'25': p25_c, '50': p50_c, '75': p75_c}
filtrado_c = {}

for name, threshold in percentiles_c.items():
    G_filt_c = nx.Graph()
    num_nodes_c = coh_df.shape[0]
    G_filt_c.add_nodes_from(range(num_nodes_c))

    for i in range(num_nodes_c):
        for j in range(i + 1, num_nodes_c):
            weight_c = coh_df.iloc[i, j]
            if abs(weight_c) > threshold:
                G_filt_c.add_edge(i, j, weight_c=weight_c)

    graph_key_c = f"coherencia espectral_{name.replace(' ', '_').lower()}"
    filtrado_c[graph_key_c] = G_filt_c

    print(f"Created graph '{graph_key_c}' with {G_filt_c.number_of_nodes()} nodes and {G_filt_c.number_of_edges()} edges.")

for graph_key, G_to_draw in filtrado_c.items():
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G_to_draw)
    nx.draw(G_to_draw, pos, with_labels=False, node_size=20, edge_color='gray', alpha=0.6)
    plt.title(f'Filtered Graph: {graph_key.replace("_", " ").title()}')
    plt.show()

In [ ]:
plt.figure(figsize=(8, 2))
plv = sns.heatmap(plv_df.values,
                 annot=False, cmap='GnBu', fmt=".2f")
plt.show()

# Distribución
plt.figure(figsize=(8, 6))
val_plv = corr_df.values[np.triu_indices_from(plv_df, k=1)]
plt.hist(val_plv, bins=45)

p25_p = np.percentile(val_plv, 25)
p50_p = np.percentile(val_plv, 50)
p75_p = np.percentile(val_plv, 75)

print(f"Percentil 25: {p25_p:.4f}")
print(f"Percentil 50: {p50_p:.4f}")
print(f"Percentil 75: {p75_p:.4f}")

# gráficas
percentiles_p = {'25': p25_p, '50': p50_p, '75': p75_p}
filtrado_p = {}

for name, threshold in percentiles_p.items():
    G_filt_p = nx.Graph()
    num_nodes_p = coh_df.shape[0]
    G_filt_p.add_nodes_from(range(num_nodes_p))

    for i in range(num_nodes_p):
        for j in range(i + 1, num_nodes_p):
            weight_p = plv_df.iloc[i, j]
            if abs(weight_p) > threshold:
                G_filt_c.add_edge(i, j, weight_p=weight_p)

    graph_key_p = f"phase locking value_{name.replace(' ', '_').lower()}"
    filtrado_p[graph_key_p] = G_filt_p

    print(f"Created graph '{graph_key_p}' with {G_filt_p.number_of_nodes()} nodes and {G_filt_p.number_of_edges()} edges.")

for graph_key, G_to_draw in filtrado_p.items():
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G_to_draw)
    nx.draw(G_to_draw, pos, with_labels=False, node_size=20, edge_color='gray', alpha=0.6)
    plt.title(f'Filtered Graph: {graph_key.replace("_", " ").title()}')
    plt.show()

# Ejercicio 3

In [ ]:

import matplotlib.cm as cm

coords = datos['Coord']

G_3d = nx.from_numpy_array(data)

degrees_3d = dict(G_3d.degree())
node_size_factor = 5
node_sizes = [degrees_3d[node] * node_size_factor for node in G_3d.nodes()]

edge_weights = np.array([G_3d[u][v]['weight'] for u, v in G_3d.edges()])

if np.max(edge_weights) != np.min(edge_weights):
    norm_edge_weights = (edge_weights - np.min(edge_weights)) / (np.max(edge_weights) - np.min(edge_weights))
else:
    norm_edge_weights = np.zeros_like(edge_weights)

cmap = cm.get_cmap('hot')
edge_colors = cmap(norm_edge_weights)

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

node_xyz = np.array([coords[i] for i in G_3d.nodes()])

ax.scatter(node_xyz[:, 0], node_xyz[:, 1], node_xyz[:, 2], s=node_sizes, c='hot', alpha=0.9)

for i, (u, v) in enumerate(G_3d.edges()):
    x = np.array([coords[u, 0], coords[v, 0]])
    y = np.array([coords[u, 1], coords[v, 1]])
    z = np.array([coords[u, 2], coords[v, 2]])
    ax.plot(x, y, z, color=edge_colors[i], alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('Gráfico 3D de la matriz de coactivación')

plt.show()

# EJEERCICIO 4

In [ ]:

import mne
raw = mne.io.read_raw_edf(r"C:/Users/rayom/OneDrive/Documentos/GitHub/Neurociencias-2026-1/S03_datasets/EEG_practica", preload=True)

print(raw)
print(raw.info)

In [ ]:
import bct

eeg_channels = [ch for ch in raw.ch_names if 'EEG' in ch];
raw_eeg = raw.copy().pick_channels(eeg_channels, ordered=True);
variances = raw_eeg.get_data().var(axis=1);
variance_threshold = 1e-10
low_variance_channels = [raw_eeg.ch_names[i] for i, var in enumerate(variances) if var < variance_threshold];
raw_eeg.drop_channels(low_variance_channels);
raw_eeg.set_eeg_reference('average', projection=True);
raw_eeg.apply_proj();
l_freq, h_freq = 1, 40
raw_eeg.filter(l_freq, h_freq, fir_design='firwin');
preprocessed_data = raw_eeg.get_data();

connectivity_matrix = np.corrcoef(preprocessed_data);
connectivity_matrix_df = pd.DataFrame(connectivity_matrix);
connectivity_matrix_df.fillna(0, inplace=True);
connectivity_matrix_df.replace([np.inf, -np.inf], 0, inplace=True);

G_edf = nx.from_numpy_array(connectivity_matrix_df.values);


# nodos
edf_degrees = dict(G_edf.degree())
print("Grado de nodos:", edf_degrees)

# Eficiencia global
edf_global_efficiency = bct.efficiency_wei(connectivity_matrix_df.values)
print("Eficiencia Global:", edf_global_efficiency)

# Eficienciea local
edf_local_efficiency = bct.efficiency_wei(connectivity_matrix_df.values, local=True)
print("Eficiencia local:", edf_local_efficiency)

# Cluster
edf_clustering_coefficient = nx.average_clustering(G_edf)
print("Coeficiente de Cluster:", edf_clustering_coefficient)

# Ruta corta
if nx.is_connected(G_edf):
    edf_avg_shortest_path_length = nx.average_shortest_path_length(G_edf)
    print("Ruta mas corta:", edf_avg_shortest_path_length)
else:
    print("Gráfica desconectada.")
    largest_component_edf = max(nx.connected_components(G_edf), key=len)
    G_edf_largest_component = G_edf.subgraph(largest_component_edf).copy()
    if len(G_edf_largest_component) > 1:
        edf_avg_shortest_path_length = nx.average_shortest_path_length(G_edf_largest_component)
        print("Promedio camino mas corto", edf_avg_shortest_path_length)
    else:
        edf_avg_shortest_path_length = 0
        print("El componente mas largo solo tiene un nodo")

# Mundo Pequeño
n_nodes_edf = G_edf.number_of_nodes()
n_edges_edf = G_edf.number_of_edges()
G_random_edf = nx.gnm_random_graph(n_nodes_edf, n_edges_edf)
clustering_coefficient_random_edf = nx.average_clustering(G_random_edf)
print("Coeficiente de cluster", clustering_coefficient_random_edf)
if nx.is_connected(G_random_edf):
    avg_shortest_path_length_random_edf = nx.average_shortest_path_length(G_random_edf)
    print("Camino mas corto", avg_shortest_path_length_random_edf)
else:
    print("La grafica esta desconectada.")
    largest_component_random_edf = max(nx.connected_components(G_random_edf), key=len)
    G_random_largest_component_edf = G_random_edf.subgraph(largest_component_random_edf).copy()
    if len(G_random_largest_component_edf) > 1:
        avg_shortest_path_length_random_edf = nx.average_shortest_path_length(G_random_largest_component_edf)
        print("Average Shortest Path Length (Random Graph - Largest Component):", avg_shortest_path_length_random_edf)
    else:
        avg_shortest_path_length_random_edf = 0
        print("Largest component of random graph has only one node. Average shortest path length is undefined.")

if avg_shortest_path_length_random_edf > 0 and edf_avg_shortest_path_length > 0 and clustering_coefficient_random_edf > 0:
    small_world_coefficient_edf = (edf_clustering_coefficient / clustering_coefficient_random_edf) / (edf_avg_shortest_path_length / avg_shortest_path_length_random_edf)
    print("Small-world Coefficient:", small_world_coefficient_edf)
elif clustering_coefficient_random_edf == 0:
     print("Clustering coefficient of random graph is zero. Cannot calculate small-world coefficient.")
else:
    print("Average shortest path length of original or random graph is zero or undefined. Cannot calculate small-world coefficient.")

# Comunidades
G_edf_abs = nx.from_numpy_array(np.abs(connectivity_matrix_df.values))
edf_partition = community_louvain.best_partition(G_edf_abs)
print("Communities:", edf_partition)

#hubs
edf_degrees_abs = dict(G_edf_abs.degree())
edf_avg_degree_abs = np.mean(list(edf_degrees_abs.values()))
edf_hubs_abs = [node for node, degree in edf_degrees_abs.items() if degree > edf_avg_degree_abs]
print("Potential Hubs (degree > average) using absolute weights:", edf_hubs_abs)

# EJERCICIO 5

In [ ]:

num_nodes = G_edf.number_of_nodes()

np.random.seed(42)
coords_edf = np.random.rand(num_nodes, 3) * 10

edf_degrees = dict(G_edf.degree())
node_size_factor = 50
node_sizes_edf = [edf_degrees[node] * node_size_factor for node in G_edf.nodes()]

edge_weights_edf = np.array([G_edf[u][v]['weight'] for u, v in G_edf.edges()])
abs_edge_weights_edf = np.abs(edge_weights_edf)


if np.max(abs_edge_weights_edf) != np.min(abs_edge_weights_edf):
    norm_edge_colors_edf = (abs_edge_weights_edf - np.min(abs_edge_weights_edf)) / (np.max(abs_edge_weights_edf) - np.min(abs_edge_weights_edf))
else:
    norm_edge_colors_edf = np.zeros_like(abs_edge_weights_edf) # All weights are the same, use one color

cmap = cm.get_cmap('hot')
edge_colors_edf = cmap(norm_edge_colors_edf)

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

node_xyz_edf = coords_edf

ax.scatter(node_xyz_edf[:, 0], node_xyz_edf[:, 1], node_xyz_edf[:, 2], s=node_sizes_edf, c='blue', alpha=0.8)

for i, (u, v) in enumerate(G_edf.edges()):
    x = np.array([coords_edf[u, 0], coords_edf[v, 0]])
    y = np.array([coords_edf[u, 1], coords_edf[v, 1]])
    z = np.array([coords_edf[u, 2], coords_edf[v, 2]])
    ax.plot(x, y, z, color=edge_colors_edf[i], alpha=0.3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('Grafico 3D de EEG de la matriz de conectividad')